In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da
import random

In [2]:
#data_dir = "/home/ibcn079/data/ukdale/house1/"
meters = {}

In [3]:
def read_channel(filename, appliance):
    """Method to read home channel data from .dat file into panda dataframe
        Args:
                filename: path to a specific channel_(m) from house(n)
        return:
                [pandas.Dataframe] of a signle channel_(m) from house(n)
    """
    channel_to_read = pd.read_csv(filename, names=["Time", appliance], delim_whitespace=True)
    return channel_to_read

def load_chan_list(app_name, ds_name='UKDALE'):
        """
        Returns corresponding meter numbers given appliance name
        For different channels with same name, it will return a list
        """
        chan_list = []
        if(ds_name=='UKDALE'):
            for line in open(os.path.join(data_dir, 'labels.dat')):
                if(line.strip('\n').split(' ')[1] == app_name or 
                   line.strip('\n').split(' ')[1][:-1] == app_name):
                    chan_list.append(int(line.strip('\n').split(' ')[0]))
        return(chan_list)

    
def load_meter(app_name, ds_name='UKDALE'):
        """
        Take an appliance name, return a list of meters object
        Each meter object is a dictionary with three attributes, appliance name,
        channel number and data which is a pandas series
        """
        data = pd.DataFrame()
        if(ds_name=='UKDALE'):
            chan_list = load_chan_list(app_name, ds_name)
            for chan_num in chan_list:
                file_name = 'channel_%d.dat' % chan_num
                df=read_channel(os.path.join(data_dir, file_name),app_name)
                df.dropna(axis=0)
                data = pd.concat([data, df])
        return data    

In [4]:
data_dir="/home/ibcn079/data/ukdale/house1/"
data = load_meter(app_name='fridge', ds_name='UKDALE')

In [5]:
len(data)
data

,Time,fridge
0,1355523692,85
1,1355523698,85
2,1355523704,84
3,1355523710,85
4,1355523716,85
5,1355523722,85
6,1355523728,85
7,1355523734,85
8,1355523740,85
9,1355523746,86


In [142]:
def generate_clean_data(path, appliance, window_size, threshold, proportion=[1,1],buildings=[1,2], test=False, test_on='All'):
    
    activation_proportion = proportion[0]
    non_activation_proportion = proportion[1]
    aggregate_channels = []
    individual_channels = []
    aggregate_channels_test = []
    individual_channels_test = []
    activations = []
    non_activations = []
    
    houses=buildings
    for house in houses:
        data_dir = path + "house" +str(house) + '/'
        print(data_dir)
        iam = load_meter(app_name=appliance, ds_name='UKDALE')
        aggregate = load_meter(app_name="aggregate", ds_name='UKDALE')
        print("Reading house: {}".format(house))
        aggregate, iaactivations = []
    non_activations = []m = np.array(aggregate['aggregate']), np.array(iam[appliance])
        
        if test:
            split = round(len(aggregate) * 0.8)
            aggregate_test = aggregate[split:]
            iam_test = iam[split:]
            if test_on == 'All':
                aggregate_channels_test.append(aggregate_test)
                individual_channels_test.append(iam_test)
                aggregate = aggregate[:split]
                iam = iam[:split]
            elif test_on == house:
                aggregate_channels_test.append(aggregate_test)
                individual_channels_test.append(iam_test)
                aggregate = aggregate[:split]
                iam = iam[:split]
                
    aggregate_channels.append(aggregate) #appending the aggregate to aggregate list and iam to iam list so that their indices match
    individual_channels.append(iam)
    
    return aggregate_channels, individual_channels   

In [101]:
path = "/home/ibcn079/data/ukdale/"
WINDOW_SIZE= 2000
Threshold =10   
appliance = 'fridge'
houses=[1]
for house in houses:
        data_dir = path + "house" +str(house) + '/'
        print(data_dir)
        iam = load_meter(app_name=appliance, ds_name='UKDALE')
        aggregate = load_meter(app_name="aggregate", ds_name='UKDALE')
        print("Reading house: {}".format(house))
        aggregate, iam = np.array(aggregate['aggregate']), np.array(iam[appliance])

/home/ibcn079/data/ukdale/house1/
Reading house: 1


In [102]:
aggregate

array([599, 582, 600, ..., 686, 667, 672])

In [143]:
agg,ind=generate_clean_data(path, appliance, window_size=WINDOW_SIZE, threshold=Threshold)

/home/ibcn079/data/ukdale/house1/
Reading house: 1
/home/ibcn079/data/ukdale/house2/
Reading house: 2


In [146]:
-

In [148]:
activations, non_activations = get_activations(individual_channels=ind, window_size=WINDOW_SIZE)

Creating activations for house : [85 85 84 ...,  0  0  0]


In [176]:
def get_dataset(aggregate_channels,individual_channels, activations, non_activations, window_size, proportion=[1,1],test=False):
    activation_proportion = proportion[0]
    non_activation_proportion = proportion[1]
    agg, iam = [], []
    for i in range(len(aggregate_channels)):
        #iterating through aggregate data of each house
        print('Number of activations in this channel: ', len(activations[i]))
        print('Number of non-activations in this channel: ', len(non_activations[i]))
        agg_windows, iam_windows = create_overlap_windows(aggregate_channels[i], individual_channels[i], window_size)
        agg.extend(agg_windows)
        iam.extend(iam_windows)
        for start, end in activations[i]:
            #then iterating through activation positions in specified house [i]
            for j in range(activation_proportion):
                #randomly generate windows #n times with one activation
                activation_size = end - start
                #randomly positioning activation in window
                print("Activation size=: {}, window size=: {}".format(activation_size, (window_size - activation_size) ))
                
                if (window_size - activation_size) >0:
                    start_aggregate = start - random.randint(0, window_size - activation_size)
                    
                if start_aggregate + window_size < len(aggregate_channels[i]):
                    agg_buff, iam_buff = aggregate_channels[i][start_aggregate: start_aggregate + window_size], individual_channels[i][start_aggregate: start_aggregate + window_size]
                    agg_buff, iam_buff = np.copy(agg_buff), np.copy(iam_buff)
                    agg.append(agg_buff)
                    iam.append(iam_buff)
        for start, end in non_activations[i]:
            for j in range(non_activation_proportion):
                window_start = random.randint(start, end - window_size)
                agg_buff, iam_buff = aggregate_channels[i][window_start: window_start + window_size], individual_channels[i][window_start: window_start + window_size]
                agg_buff, iam_buff = np.copy(agg_buff), np.copy(iam_buff)
                agg.append(agg_buff)
                iam.append(iam_buff)
    zipper = list(zip(agg, iam))
    random.shuffle(zipper)
    agg, iam = zip(*zipper)
    agg, iam = np.array(agg), np.array(iam)
    dataset = [agg, iam]

    #Creating test set if test==True
    agg_test = []
    iam_test = []
    isFirst = True
    if test:
        for i in range(len(aggregate_channels_test)):
            agg_buff_test, iam_buff_test = create_overlap_windows(aggregate_channels_test[i], individual_channels_test[i], window_size=window_size)
            if isFirst:
                agg_test = agg_buff_test
                iam_test = iam_buff_test
                isFirst = False
            else:
                print(agg_test)
                print(agg_buff_test)
                agg_test = np.concatenate((agg_test, agg_buff_test), axis=0)
                iam_test = npcreate_windows.concatenate((iam_test, iam_buff_test), axis=0)
        testset = [agg_test, iam_test]
        return dataset, testset

    return dataset

In [173]:
def create_overlap_windows(agg, iam, window_size, stride = 10):
    position = 0
    agg_windows = []
    iam_windows = []
    while position < len(agg) - window_size -1:
        agg_buffer = agg[position: position + window_size]
        iam_buffer = iam[position: position + window_size]
        agg_windows.append(agg_buffer)
        iam_windows.append(iam_buffer)
        position += stride
    return agg_windows, iam_windows

In [178]:
dataset = get_dataset(agg,ind, activations, non_activations, 2000, test=False)

Number of activations in this channel:  8238200
Number of non-activations in this channel:  0
Activation size=: 457, window size=: 1543
Activation size=: 456, window size=: 1544
Activation size=: 455, window size=: 1545
Activation size=: 454, window size=: 1546
Activation size=: 453, window size=: 1547
Activation size=: 452, window size=: 1548
Activation size=: 451, window size=: 1549
Activation size=: 450, window size=: 1550
Activation size=: 449, window size=: 1551
Activation size=: 448, window size=: 1552
Activation size=: 447, window size=: 1553
Activation size=: 446, window size=: 1554
Activation size=: 445, window size=: 1555
Activation size=: 444, window size=: 1556
Activation size=: 443, window size=: 1557
Activation size=: 442, window size=: 1558
Activation size=: 441, window size=: 1559
Activation size=: 440, window size=: 1560
Activation size=: 439, window size=: 1561
Activation size=: 438, window size=: 1562
Activation size=: 437, window size=: 1563
Activation size=: 436, w

Activation size=: 171, window size=: 1829
Activation size=: 170, window size=: 1830
Activation size=: 169, window size=: 1831
Activation size=: 168, window size=: 1832
Activation size=: 167, window size=: 1833
Activation size=: 166, window size=: 1834
Activation size=: 165, window size=: 1835
Activation size=: 164, window size=: 1836
Activation size=: 163, window size=: 1837
Activation size=: 162, window size=: 1838
Activation size=: 161, window size=: 1839
Activation size=: 160, window size=: 1840
Activation size=: 159, window size=: 1841
Activation size=: 158, window size=: 1842
Activation size=: 157, window size=: 1843
Activation size=: 156, window size=: 1844
Activation size=: 155, window size=: 1845
Activation size=: 154, window size=: 1846
Activation size=: 153, window size=: 1847
Activation size=: 152, window size=: 1848
Activation size=: 151, window size=: 1849
Activation size=: 150, window size=: 1850
Activation size=: 149, window size=: 1851
Activation size=: 148, window size

Activation size=: 12, window size=: 1988
Activation size=: 11, window size=: 1989
Activation size=: 10, window size=: 1990
Activation size=: 9, window size=: 1991
Activation size=: 8, window size=: 1992
Activation size=: 7, window size=: 1993
Activation size=: 6, window size=: 1994
Activation size=: 5, window size=: 1995
Activation size=: 4, window size=: 1996
Activation size=: 3, window size=: 1997
Activation size=: 2, window size=: 1998
Activation size=: 1, window size=: 1999
Activation size=: 111, window size=: 1889
Activation size=: 110, window size=: 1890
Activation size=: 109, window size=: 1891
Activation size=: 108, window size=: 1892
Activation size=: 107, window size=: 1893
Activation size=: 106, window size=: 1894
Activation size=: 105, window size=: 1895
Activation size=: 104, window size=: 1896
Activation size=: 103, window size=: 1897
Activation size=: 102, window size=: 1898
Activation size=: 101, window size=: 1899
Activation size=: 100, window size=: 1900
Activation si

Activation size=: 197, window size=: 1803
Activation size=: 196, window size=: 1804
Activation size=: 195, window size=: 1805
Activation size=: 194, window size=: 1806
Activation size=: 193, window size=: 1807
Activation size=: 192, window size=: 1808
Activation size=: 191, window size=: 1809
Activation size=: 190, window size=: 1810
Activation size=: 189, window size=: 1811
Activation size=: 188, window size=: 1812
Activation size=: 187, window size=: 1813
Activation size=: 186, window size=: 1814
Activation size=: 185, window size=: 1815
Activation size=: 184, window size=: 1816
Activation size=: 183, window size=: 1817
Activation size=: 182, window size=: 1818
Activation size=: 181, window size=: 1819
Activation size=: 180, window size=: 1820
Activation size=: 179, window size=: 1821
Activation size=: 178, window size=: 1822
Activation size=: 177, window size=: 1823
Activation size=: 176, window size=: 1824
Activation size=: 175, window size=: 1825
Activation size=: 174, window size

Activation size=: 90, window size=: 1910
Activation size=: 89, window size=: 1911
Activation size=: 88, window size=: 1912
Activation size=: 87, window size=: 1913
Activation size=: 86, window size=: 1914
Activation size=: 85, window size=: 1915
Activation size=: 84, window size=: 1916
Activation size=: 83, window size=: 1917
Activation size=: 82, window size=: 1918
Activation size=: 81, window size=: 1919
Activation size=: 80, window size=: 1920
Activation size=: 79, window size=: 1921
Activation size=: 78, window size=: 1922
Activation size=: 77, window size=: 1923
Activation size=: 76, window size=: 1924
Activation size=: 75, window size=: 1925
Activation size=: 74, window size=: 1926
Activation size=: 73, window size=: 1927
Activation size=: 72, window size=: 1928
Activation size=: 71, window size=: 1929
Activation size=: 70, window size=: 1930
Activation size=: 69, window size=: 1931
Activation size=: 68, window size=: 1932
Activation size=: 67, window size=: 1933
Activation size=

...

Activation size=: 291, window size=: 1709
Activation size=: 290, window size=: 1710
Activation size=: 289, window size=: 1711
Activation size=: 288, window size=: 1712
Activation size=: 287, window size=: 1713
Activation size=: 286, window size=: 1714
Activation size=: 285, window size=: 1715
Activation size=: 284, window size=: 1716
Activation size=: 283, window size=: 1717
Activation size=: 282, window size=: 1718
Activation size=: 281, window size=: 1719
Activation size=: 280, window size=: 1720
Activation size=: 279, window size=: 1721
Activation size=: 278, window size=: 1722
Activation size=: 277, window size=: 1723
Activation size=: 276, window size=: 1724
Activation size=: 275, window size=: 1725
Activation size=: 274, window size=: 1726
Activation size=: 273, window size=: 1727
Activation size=: 272, window size=: 1728
Activation size=: 271, window size=: 1729
Activation size=: 270, window size=: 1730
Activation size=: 269, window size=: 1731
Activation size=: 268, window size

Activation size=: 52, window size=: 1948
Activation size=: 51, window size=: 1949
Activation size=: 50, window size=: 1950
Activation size=: 49, window size=: 1951
Activation size=: 48, window size=: 1952
Activation size=: 47, window size=: 1953
Activation size=: 46, window size=: 1954
Activation size=: 45, window size=: 1955
Activation size=: 44, window size=: 1956
Activation size=: 43, window size=: 1957
Activation size=: 42, window size=: 1958
Activation size=: 41, window size=: 1959
Activation size=: 40, window size=: 1960
Activation size=: 39, window size=: 1961
Activation size=: 38, window size=: 1962
Activation size=: 37, window size=: 1963
Activation size=: 36, window size=: 1964
Activation size=: 35, window size=: 1965
Activation size=: 34, window size=: 1966
Activation size=: 33, window size=: 1967
Activation size=: 32, window size=: 1968
Activation size=: 31, window size=: 1969
Activation size=: 30, window size=: 1970
Activation size=: 29, window size=: 1971
Activation size=

Activation size=: 198, window size=: 1802
Activation size=: 197, window size=: 1803
Activation size=: 196, window size=: 1804
Activation size=: 195, window size=: 1805
Activation size=: 194, window size=: 1806
Activation size=: 193, window size=: 1807
Activation size=: 192, window size=: 1808
Activation size=: 191, window size=: 1809
Activation size=: 190, window size=: 1810
Activation size=: 189, window size=: 1811
Activation size=: 188, window size=: 1812
Activation size=: 187, window size=: 1813
Activation size=: 186, window size=: 1814
Activation size=: 185, window size=: 1815
Activation size=: 184, window size=: 1816
Activation size=: 183, window size=: 1817
Activation size=: 182, window size=: 1818
Activation size=: 181, window size=: 1819
Activation size=: 180, window size=: 1820
Activation size=: 179, window size=: 1821
Activation size=: 178, window size=: 1822
Activation size=: 177, window size=: 1823
Activation size=: 176, window size=: 1824
Activation size=: 175, window size

Activation size=: 207, window size=: 1793
Activation size=: 206, window size=: 1794
Activation size=: 205, window size=: 1795
Activation size=: 204, window size=: 1796
Activation size=: 203, window size=: 1797
Activation size=: 202, window size=: 1798
Activation size=: 201, window size=: 1799
Activation size=: 200, window size=: 1800
Activation size=: 199, window size=: 1801
Activation size=: 198, window size=: 1802
Activation size=: 197, window size=: 1803
Activation size=: 196, window size=: 1804
Activation size=: 195, window size=: 1805
Activation size=: 194, window size=: 1806
Activation size=: 193, window size=: 1807
Activation size=: 192, window size=: 1808
Activation size=: 191, window size=: 1809
Activation size=: 190, window size=: 1810
Activation size=: 189, window size=: 1811
Activation size=: 188, window size=: 1812
Activation size=: 187, window size=: 1813
Activation size=: 186, window size=: 1814
Activation size=: 185, window size=: 1815
Activation size=: 184, window size

ValueError: empty range for randrange() (0,-563, -563)